In [21]:
import math 
import matplotlib.dates as mdate
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

psi = 2
gamma = 1

def f(x):
    return math.sin(math.pi*x) + x*(1-x)

def _lambda(gamma,k,h):
    return gamma*(k/h**2)

p = 0
q = 0

## Método explícito

In [25]:
h = 0.01 # Valor do passo em x

#k_stable = 0.00001 # lambda == 0.1

#k_instable = 0.0001 # lambda == 1

#lambd = _lambda(gamma,k,h) # Valor de lambda

m = int(1/h) # Quantidade de posicoes x na barra

A = np.zeros((5,5))

#for i in range(5):
#    A.itemset((i,i),10)
    
print(matrix)


[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
